steps to predict the output:
1. load the model
2. load all the pickle files
3. using the pickle files , transform the input data into the required format
4. means one hot encoding the geography column , lable encoding the gender column of the input
5. the using the scaler.pkl file , scale the input data
6. predict the output using the model

In [14]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

In [ ]:
#load the model
model = load_model('model.h5')
model

In [6]:
#load the pic
with open('label_encoder_gender.pkl','rb') as f:
    label_encoder_gender = pickle.load(f)

with open('onehotencoder_geography.pkl','rb') as f:
    onehotencoder_geography = pickle.load(f)
    
with open('scaler.pkl','rb') as f:
    scaler = pickle.load(f)
    

In [8]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [20]:
input_df = pd.DataFrame([input_data])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [21]:
# transform the Gender
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [23]:

# One-hot encode 'Geography'
geo_encoded = onehotencoder_geography.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehotencoder_geography.get_feature_names_out(['Geography']))
geo_encoded_df


/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [24]:
#concatinate the data
input_df = pd.concat([input_df.drop('Geography',axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [ ]:
scaled_input  = scaler.transform(input_df)
output = model.predict(scaled_input) 
print(output)
if output > 0.5:
    print('The customer is likely to leave the bank')
else:
    print('The customer is likely to stay with the bank')

1/1 [==============================] - 0s 31ms/step
[[0.10374889]]
The customer is likely to stay with the bank


In [25]:
# shortcut
model.predict(scaler.transform(input_df)) > 0.5

1/1 [==============================] - 0s 237ms/step


array([[False]])